In [2]:
import pandas as pd
import numpy as np
import utils
import Dotblot
import GeneralDilution

from importlib import reload
import os
import re
import json

In [14]:
a = "test"
b = 2

if a == "test" and (b == 2 or b == 3):
    print("yesssssss")

yesssssss


In [7]:
utils.dilution_position_def("Pos_Ctr_Vial", 3, 2)

(array(['Pos_Ctr_Vial[003]', 'Pos_Ctr_Vial[004]'], dtype='<U32'),
 array([1, 1]))

### dotblot

In [1]:
reload(utils)
reload(Dotblot)
dotblot_method = Dotblot.DotblotMethod()

normal_dotblot_file = r"L:/Departements/BTDS_AD/002_AFFS/Lab Automation/09. Tecan/06. DotBlot_automation_DPP/DotBlot automation dilution data.xlsx"
_2_coating_dotblot_file = r"L:/Departements/BTDS_AD/002_AFFS/Lab Automation/09. Tecan/06. DotBlot_automation_DPP/DotBlot automation dilution data - 2 coating.xlsx"

# data = utils.import_excel_dotblot(normal_dotblot_file)
data = utils.import_excel_dotblot_2_coating(_2_coating_dotblot_file)
sample_dilution_data = data[0]
coating_protein_dilution_data = data[1]
pos_control_dilution_data = data[2]
neg_control_dilution_data = data[3]

NameError: name 'reload' is not defined

In [26]:
pos_control_dilution_data[0]

,Dilution step,Initial concentration,Withdrawn volume,Final concentration,Assay buffer volume,Sample volume,Final Volume
20,1,10.2,6,0.08,759.0,6,765.0
21,2,0.08,10,0.002,390,10,400


In [27]:
pos_control_dilution_data[1]

,Dilution step,Initial concentration,Withdrawn volume,Final concentration,Assay buffer volume,Sample volume,Final Volume
27,1,10.2,6,0.08,759.0,6,765.0
28,2,0.08,10,0.002,390,10,400
29,3,0.002,100,0.0002,900,100,1000


In [52]:
reload(Dotblot)
dotblot_method = Dotblot.DotblotMethod(debug=False)

# Reset parameters

dotblot_method.last_eppendorf_pos = 1
dotblot_method.last_deep_well_pos = 1

dotblot_method.n_pos_control_steps = len(pos_control_dilution_data[0]["Assay buffer volume"])
dotblot_method.n_neg_control_steps = len(neg_control_dilution_data[0]["Assay buffer volume"])
dotblot_method.sample_dilution_data = [sample_dilution_data[i].to_dict(orient="list") for i in range(len(sample_dilution_data))]
dotblot_method.pos_control_dilution_data = [pos_control_dilution_data[i].to_dict(orient="list") for i in range(len(pos_control_dilution_data))]
dotblot_method.neg_control_dilution_data = [neg_control_dilution_data[i].to_dict(orient="list") for i in range(len(neg_control_dilution_data))]
dotblot_method.pos_control_vial_posX = 1
dotblot_method.pos_control_vial_posY = 1
dotblot_method.main_sample_labware_type = "FakeFalcon15"
dotblot_method.main_sample_dil_destination = "DeepWell"
dotblot_method.n_samples_main_dilution = 1
dotblot_method.n_sample_dilution_steps = len(sample_dilution_data[0]["Assay buffer volume"])
dotblot_method.pump_steps_data = [
                        {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Coating protein"}, 
                        {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Pos/Neg control"}, 
                        {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Samples"}
                    ]

dotblot_method.dotblot()

pos dilutions done: [[1], [2]]
negative ctr eppendorf pos: [[3], [4]]
sample eppendorf pos: [[6], [7]]
calculated pump labware positions
transfer volume instruction  1 done
transfer volume instruction  2 done
transfer volume instruction  3 done
Dye and wash files generated.
generated all GWL files


([[1], [2]], [[3], [4]], [[6], [7]])

In [ ]:
csv_path = r'L:\Departements\BTDS_AD\002_AFFS\Lab Automation\09. Tecan\01. Methods\3. DotBlot\2. Positive control.csv'

mix_data = []
labware_name = "Custom holder[001]"
labware_pos = 1
volume = 300

for i in range(3):
    mix_data.append([labware_name, labware_pos, labware_name, labware_pos, volume])

mix_data
mix_csv = pd.DataFrame(mix_data, columns=None)
mix_csv

original_csv = pd.read_csv(csv_path, header=None) # create dataframe from original CSV file
final_csv = pd.concat([original_csv, mix_csv], ignore_index=True) # concatenate both dataframes into one
final_csv.to_csv(csv_path, index=False, header=False)

In [ ]:
pump_step_data_simple = [{'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Vacuum'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}]
pump_step_data = [{'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Samples', 'wells_type': 'All wells'}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Coating protein', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'Blocking buffer', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Pos/Neg control', 'wells_type': 'Only samples'}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Conjugate', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Vacuum'}]

pump_step_data

### For Dotblot first try for TV-TMD-35372

In [ ]:
steps = [
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Coating protein'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'Blocking buffer'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Pos/Neg control'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Samples'}, 
         {'step_type': 'Wait timer', 'wait_timer': 600}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Conjugate'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'},  
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'},  
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'}
]

steps

In [ ]:
dotblot_method.pump_steps_data = pump_step_data
dotblot_method.sample_eppendorf_positions = [3,4,5,6]

dotblot_method.generate_pump_step_instruction_files()

In [ ]:
dotblot_method.transfer_volume_to_wells(dotblot_method.dpbs_lw_name, "96 Deep Well[001]", 100)

#### GENERAL DILUTION TESTS

In [ ]:
reload(utils)
reload(GeneralDilution)

gen_dil = GeneralDilution.GeneralDilution()

In [ ]:
sample_dilution_data = {'Dilution step': [1.0, 2.0, 3.0], 'Initial concentration': [150.0, 0.7, 0.003], 'Withdrawn volume': [4.0, 4.0, 30.0], 'Final concentration': [0.7, 0.003, 0.0001], 'Assay buffer volume': [853.1428571428572, 929.3333333333333, 869.9999999999999], 'Sample volume': [4.0, 4.0, 30.0], 'Final Volume': [857.1428571428572, 933.3333333333333, 899.9999999999999]}
gen_dil.sample_dilution_data = sample_dilution_data
gen_dil.n_sample_dilution_steps = len(gen_dil.sample_dilution_data["Assay buffer volume"])
gen_dil.n_samples = 7
gen_dil.buffer_type = "DPBS"
gen_dil.sample_lw_origin = "Falcon15"
gen_dil.sample_lw_dest = "2R Vial"
gen_dil.used_labware_pos = dict.fromkeys(gen_dil.last_labware_pos, 0)

In [ ]:
gen_dil.general_dilution()

In [ ]:
gen_dil.used_labware_pos

In [ ]:
gen_dil.next_labware_pos(utils.LabwareNames["Falcon50"])

### VOLUME TRANSFER

In [ ]:
import VolumeTransfer
from importlib import reload
from utils import *

In [ ]:
reload(VolumeTransfer)
reload(utils)
vol_tr = VolumeTransfer.VolumeTransfer()

In [ ]:
VolumeTransfer.dilution_position_def()

In [ ]:
vol_tr.sample_lw_origin = "Eppendorf"
vol_tr.n_samples = 3
vol_tr.sample_lw_dest = "384_Well"
vol_tr.n_sample_repetitions = 4
vol_tr.volume_transfered = 20
vol_tr.used_labware_pos = dict.fromkeys(vol_tr.used_labware_pos, 0) # reset dict

vol_tr.volume_transfer()

In [ ]:
vol_tr.used_labware_pos

### Calculate labware type for each reagent based on volumes used in the method

In [ ]:
reload(utils)

In [ ]:
step_types = [
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60},
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Coating protein"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "Blocking buffer"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Pos/Neg control"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Samples"}, 
                {"step_type": "Wait timer", "wait_timer": 600}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Conjugate"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"},  
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"},  
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"}]

REAGENTS = ["samples", "pos_ctr", "neg_ctr", "dpbs", "conjugate", "blocking_buffer", "coating_protein"]
n_samples = 4

In [ ]:
def calculate_total_volumes(step_types):
    total_volume = {}

    for step in step_types:
        if step["step_type"] == "Transfer volume to wells":
            if step["liquid_type"] not in total_volume:
                total_volume[step["liquid_type"]] = 0 # create key
            
            total_volume[step["liquid_type"]] = total_volume[step["liquid_type"]] + float(step["volume_amount"])/1000 # because volume amount is in uL and we want it in mL

    # del(total_volume["Pos/Neg control"]) # remove this key because it is not needed
    total_volume["Pos_ctr"] = total_volume.pop("Pos/Neg control") # change name of key
    total_volume["Neg_ctr"] = total_volume["Pos_ctr"] # copy the value into a new key

    # multiply by number of samples
    for key in total_volume:
        total_volume[key] *= n_samples
        total_volume[key] *= 3 # because each sample is triplicated (placed in 3 wells)
        total_volume[key] = round(total_volume[key], 2)


    pos_ctr_sample_vol = round(pos_control_dilution_data["Sample volume"].sum(), 1)
    neg_ctr_sample_vol = round(neg_control_dilution_data["Sample volume"].sum(), 1)
    buffer_vol = round(pos_control_dilution_data["Assay buffer volume"].sum(), 2) + round(neg_control_dilution_data["Assay buffer volume"].sum(), 2)
    
    total_volume["Pos_ctr"] = total_volume["Pos_ctr"] + pos_ctr_vol
    total_volume["Neg_ctr"] = total_volume["Neg_ctr"] + neg_ctr_vol
    # if buffer type is DPBS or Assay buffer, choose correct key name
    buffer_type = "DPBS"
    total_volume[buffer_type] = total_volume[buffer_type] + buffer_vol

In [ ]:
total_volume = {}

for step in step_types:
    if step["step_type"] == "Transfer volume to wells":
        if step["liquid_type"] not in total_volume:
            total_volume[step["liquid_type"]] = 0 # create key
        
        total_volume[step["liquid_type"]] = total_volume[step["liquid_type"]] + float(step["volume_amount"])/1000 # because volume amount is in uL and we want it in mL

# del(total_volume["Pos/Neg control"]) # remove this key because it is not needed
total_volume["Pos_ctr"] = total_volume.pop("Pos/Neg control") # change name of key
total_volume["Neg_ctr"] = total_volume["Pos_ctr"] # copy the value into a new key

total_volume

In [ ]:
for key in total_volume:
    total_volume[key] *= n_samples
    total_volume[key] *= 3 # because each sample is triplicated (placed in 3 wells)
    total_volume[key] = round(total_volume[key], 2)

total_volume

In [ ]:
def calculate_pos_neg_ctr_volumes(pos_control_dilution_data: dict, neg_control_dilution_data: dict) -> tuple:
    pos_ctr_sample_vol = round(pos_control_dilution_data["Sample volume"].sum(), 1)
    neg_ctr_sample_vol = round(neg_control_dilution_data["Sample volume"].sum(), 1)
    buffer_vol = round(pos_control_dilution_data["Assay buffer volume"].sum(), 2) + round(neg_control_dilution_data["Assay buffer volume"].sum(), 2)

    return (pos_ctr_sample_vol, neg_ctr_sample_vol, buffer_vol)


In [ ]:
pos_ctr_vol, neg_ctr_vol, buffer_vol = calculate_pos_neg_ctr_volumes(pos_control_dilution_data, neg_control_dilution_data)

In [ ]:
total_volume["Pos_ctr"] = total_volume["Pos_ctr"] + pos_ctr_vol
total_volume["Neg_ctr"] = total_volume["Neg_ctr"] + neg_ctr_vol
# if buffer type is DPBS or Assay buffer, choose correct key name
buffer_type = "DPBS"
total_volume[buffer_type] = total_volume[buffer_type] + buffer_vol

total_volume

In [ ]:
def find_best_container(volumes_dict, containers_dict):
    result = {}
    
    for item, volume_needed in volumes_dict.items():
        best_container = None
        best_sum = float('inf')
        
        for container, (vol, cap) in containers_dict.items():
            if volume_needed <= cap - vol:
                current_sum = vol + cap
                if current_sum < best_sum:
                    best_sum = current_sum
                    best_container = container
        
        if best_container:
            result[item] = best_container
        else:
            result[item] = "No suitable container"
    
    return result

In [ ]:
def find_best_container(reagents_vol_dict: dict, labware_dict: dict):
    """
    Assigns the best labware type for each reagent considering the volume needed.
    """

    result = {}
    
    for item, volume_needed in reagents_vol_dict.items():
        best_container = None
        best_sum = float('inf')
        
        for labware, (dead_vol, max_vol) in labware_dict.items():
            if volume_needed <= max_vol - dead_vol:
                current_sum = max_vol - dead_vol
                if current_sum < best_sum:
                    best_sum = current_sum
                    best_container = labware
        
        if best_container:
            result[item] = best_container
        else:
            result[item] = "VOLUME TOO BIG"
    
    return result

In [ ]:
total_volume["Conjugate"] = 14201

find_best_container(total_volume, LABWARE_INFO)

In [ ]:
utils.find_best_container(0.2, utils.LABWARE_INFO)

In [ ]:
reload(utils)

### nanoDSF

In [ ]:
import nanoDSF

In [ ]:
reload(nanoDSF)
reload(utils)

nanomethod = nanoDSF.nanoDSFMethod()

In [ ]:
nanomethod.n_samples = 4 # amount of samples for the sample transfer
nanomethod.sample_volume_per_well = 26 # volume (uL) to transfer to each well
nanomethod.sample_lw_origin = "Eppendorf" # origin labware of samples
nanomethod.sample_triplicates = True

nanomethod.nanoDSF()

### A280

In [ ]:
import A280

In [ ]:
reload(utils)
reload(A280)

In [ ]:
utils.find_best_container(10)

In [ ]:
a280_method = A280.A280Method()

a280_method.n_samples = 4
a280_method.sample_concentration = 150
a280_method.sample_lw_origin = "Falcon15"

a280_method.a280()

In [ ]:
LABWARE_DEAD_VOLUMES

In [ ]:
LABWARE_DEAD_VOLUMES.Eppendorf

In [ ]:
def find_best_container(reagents: dict | float | int):
    """
    Assigns the best labware type for each reagent considering the volume needed.

    """


    if isinstance(reagents, (float | int)):
        volume_needed = reagents
        best_container = None
        best_sum = float('inf')

        for labware, (dead_vol, max_vol) in utils.LABWARE_DEAD_VOLUMES.to_dict().items():
            if volume_needed <= max_vol - dead_vol:
                current_sum = max_vol - dead_vol
                if current_sum < best_sum:
                    best_sum = current_sum
                    best_container = labware

        if best_container:
            return best_container
        else:
            return "VOLUME TOO BIG"

    if isinstance(reagents, dict):
        result = {}
        
        for item, volume_needed in reagents.items():
            best_container = None
            best_sum = float('inf')
            
            for labware, (dead_vol, max_vol) in utils.LABWARE_DEAD_VOLUMES.to_dict().items():
                if volume_needed <= max_vol - dead_vol:
                    current_sum = max_vol - dead_vol
                    if current_sum < best_sum:
                        best_sum = current_sum
                        best_container = labware
            
            if best_container:
                result[item] = best_container
            else:
                result[item] = "VOLUME TOO BIG"
        
        return result

In [ ]:
find_best_container(10)

### testssssstststststs

In [ ]:
pos = [i for i in range(1, 10)]

In [ ]:
reload(utils)

In [ ]:
utils.get_deep_well_pos(9, 384, "horizontal")